# Model answers to CVE154 Laboratory Activity 1

*Last updated by Christian Cahig on 20 October 2024*

In [1]:
import math as mt
import scipy.optimize as spopt

## Analysis

Since the vehicle is to move at a constant velocity,
the vector sum of the four cable forces is zero.
So, the force balance equations in terms of eastward (horizontal) and of northward (vertical) components are

$$
F_{1} \cos\theta_{1} + F_{2} \sin\theta_{2} + L = 0,
\\
-F_{1} \sin\theta_{1} + F_{2} \cos\theta_{2} + M = 0,
$$

where

$$
L = F_{3} \cos\theta_{3} - \frac{r_{w}}{\sqrt{r_{w}^{2} + r_{s}^{2}}} F_{4},
\\
M = F_{3} \sin\theta_{3} - \frac{r_{s}}{\sqrt{r_{w}^{2} + r_{s}^{2}}} F_{4}.
$$

In [2]:
F1 = 4.0
F3 = 5.0
F4 = 7.0
theta2 = 70.0
theta3 = 30.0
rs = 3.0
rw = 4.0
L = (F3 * mt.cos(mt.radians(theta3))) - ((rw * F4) / mt.sqrt((rs**2) + (rw**2)))
M = (F3 * mt.sin(mt.radians(theta3))) - (((rs * F4)) / mt.sqrt((rs**2) + (rw**2)))

## Part 1. Using bisection to solve for $F_2$

We can rewrite the force balance equations as

$$
F_{1} \cos\theta_{1} = -F_{2} \sin\theta_{2} - L,
\quad \quad
-F_{1} \sin\theta_{1} = -F_{2} \cos\theta_{2} - M.
$$

Squaring both sides of the equations,
taking the sum of the corresponding sides,
and
invoking the Pythagorean trigonometric identity,
we have

$$
F_{1}^{2} = F_{2}^{2} + 2N F_{2} + L^{2} + M^{2},
\quad\quad
N = L\sin\theta_{2} + M\cos\theta_{2}.
$$

Therefore,

$$
f\!\left(x\right) = 
x^{2} + 2N x + L^{2} + M^{2} - F_{1}^{2}.
$$

In [3]:
N = (L * mt.sin(mt.radians(theta2))) + (M * mt.cos(mt.radians(theta2)))
def residual_from_F2(x):
    return (x**2) + (2*N*x) + (L**2) + (M**2) - (F1**2)

In [4]:
XL_1, XU_1 = 0, 100
XL_2, XU_2 = -100, 0
X_TOL = 1e-7

In [5]:
print(f"Finding a root in the interval [{XL_1}, {XU_1}]...")
X_1, X_1_info = spopt.bisect(
    residual_from_F2, XL_1, XU_1,
    xtol = X_TOL,   # It's fine if you use the keyword argument `rtol`.
    maxiter = 1000,
    full_output = True, disp = False
)
print(X_1_info)
print(f"Residual value: {residual_from_F2(X_1)}")

print(f"Finding a root in the interval [{XL_2}, {XU_2}]...")
X_2, X_2_info = spopt.bisect(
    residual_from_F2, XL_2, XU_2,
    xtol = X_TOL,   # It's fine if you use the keyword argument `rtol`.
    maxiter = 1000,
    full_output = True, disp = False
)
print(X_2_info)
print(f"Residual value: {residual_from_F2(X_2)}")

Finding a root in the interval [0, 100]...
      converged: True
           flag: converged
 function_calls: 32
     iterations: 30
           root: 5.601873528212309
         method: bisect
Residual value: -1.2778023261716953e-07
Finding a root in the interval [-100, 0]...
      converged: True
           flag: converged
 function_calls: 32
     iterations: 30
           root: -2.052424568682909
         method: bisect
Residual value: 3.8571837990275526e-07


The force balance equations give us two ways
to compute $\theta_{1}$ for a given $F_{2}$.
One uses $\arccos$ (*i.e.*, from the sum of eastward components),
while the other uses $\arcsin$ (*i.e.*, from the sum of northward components):

$$
\theta_{1} = \arccos\frac{-F_{2} \sin\theta_{2} - L}{F_{1}},
\quad \quad \quad
\theta_{1} = \arcsin\frac{F_{2} \cos\theta_{2} + M}{F_{1}}.
$$

We can create two Python functions implementing these two approaches, say,

```python
def theta1_from_F2_horz(x):
    return mt.acos(-((x * mt.sin(mt.radians(theta2))) + L) / F1)

def theta1_from_F2_vert(x):
    return mt.asin(((x * mt.cos(mt.radians(theta2))) + M) / F1)
```

This is a fine strategy,
although it leaves efficiency to one's desire.

A more succinct strategy is to use keyword arguments
so that a single Python function
- allows us to specify which approach to use,
- allows us to specify if the $\theta_{1}$ is to be in radians or in degrees,
  and
- assumes a default procedure if no specifications are given.


In [6]:
def theta1_from_F2(
    x,
    use_E_force_balance = True,
    use_degrees = True,
):
    if use_E_force_balance:
        # Compute theta_1 via force balance of eastward components
        theta1 = mt.acos(-((x * mt.sin(mt.radians(theta2))) + L) / F1)
    else:
        # Compute theta_1 via force balance of northward components
        theta1 = mt.asin(((x * mt.cos(mt.radians(theta2))) + M) / F1)

    return mt.degrees(theta1) if use_degrees else theta1

In [7]:
print(
    f"If the second cable pulls with a {X_1}-kN force,",
    "the first cable pulls along the direction of:\n"
    f"\t{theta1_from_F2(X_1, use_E_force_balance=True)}° S of E",
    "(according to the balance of eastward force components)\n"
    f"\t{theta1_from_F2(X_1, use_E_force_balance=False)}° S of E",
    "(according to the balance of northward force components)"
)

print(
    f"If the second cable pulls with a {X_2}-kN force,",
    "the first cable pulls along the direction of:\n"
    f"\t{theta1_from_F2(X_2, use_E_force_balance=True)}° S of E",
    "(according to the balance of eastward force components)\n"
    f"\t{theta1_from_F2(X_2, use_E_force_balance=False)}° S of E",
    "(according to the balance of northward force components)"
)

If the second cable pulls with a 5.601873528212309-kN force, the first cable pulls along the direction of:
	176.90518380367482° S of E (according to the balance of eastward force components)
	3.0948119523830204° S of E (according to the balance of northward force components)
If the second cable pulls with a -2.052424568682909-kN force, the first cable pulls along the direction of:
	36.905186836161995° S of E (according to the balance of eastward force components)
	-36.90518827445071° S of E (according to the balance of northward force components)


As an aside, it may be a good rule of thumb to prefer $\arccos$ over $\arcsin$;
see [this StackExchange answer](https://math.stackexchange.com/a/4860244).
On the interval $0 \leq \vartheta < \pi$,
the sine function is not injective,
which means that
$\sin\vartheta_{1} = \sin\vartheta_{2}$
even when $\vartheta_{1} \neq \vartheta_{2}$.

## Part 2. Using bisection to solve for $\theta_1$


We can rewrite the force balance equations as two expressions of $F_{2}$,

$$
F_{2}
= \frac{-F_{1}\cos\theta_{1} - L}{\sin\theta_{2}},
\quad \quad \quad
F_{2}
= \frac{F_{1}\sin\theta_{1} - M}{\cos\theta_{2}},
$$

*i.e.*,

$$
O\cos\theta_{1} + P\sin\theta_{1} + Q = 0,
$$

where

$$
O = \frac{F_{1}}{\sin\theta_{2}},
\quad
P = \frac{F_{1}}{\cos\theta_{2}},
\quad
Q = \frac{L}{\sin\theta_{2}} - \frac{M}{\cos\theta_{2}}.
$$

Therefore,

$$
g\!\left(z\right) = O \cos z + P \sin z + Q.
$$

In [8]:
O = F1 / mt.sin(mt.radians(theta2))
P = F1 / mt.cos(mt.radians(theta2))
Q = (L / mt.sin(mt.radians(theta2))) - (M / mt.cos(mt.radians(theta2)))

def residual_from_theta1(z):
    # Note that `z` is in radians so we can deal with "smaller" values
    return (O * mt.cos(z)) + (P * mt.sin(z)) + Q

In [9]:
ZL_1, ZU_1 = 0, mt.pi
ZL_2, ZU_2 = -mt.pi, 2*mt.pi
Z_TOL = 1e-7

In [10]:
print(f"Finding a root in the interval [{ZL_1}, {ZU_1}]...")
Z_1, Z_1_info = spopt.bisect(
    residual_from_theta1, ZL_1, ZU_1,
    xtol = Z_TOL,   # It's fine if you use the keyword argument `rtol`.
    maxiter = 1000,
    full_output = True, disp = False
)
print(Z_1_info)
print(f"Residual value: {residual_from_theta1(Z_1)}")

print(f"Finding a root in the interval [{ZL_2}, {ZU_2}]...")
Z_2, Z_2_info = spopt.bisect(
    residual_from_theta1, ZL_2, ZU_2,
    xtol = Z_TOL,   # It's fine if you use the keyword argument `rtol`.
    maxiter = 1000,
    full_output = True, disp = False
)
print(Z_2_info)
print(f"Residual value: {residual_from_theta1(Z_2)}")

Finding a root in the interval [0, 3.141592653589793]...
      converged: True
           flag: converged
 function_calls: 27
     iterations: 25
           root: 3.0875779730329667
         method: bisect
Residual value: 2.4802130305090486e-07
Finding a root in the interval [-3.141592653589793, 6.283185307179586]...
      converged: True
           flag: converged
 function_calls: 29
     iterations: 27
           root: -0.6441170020357027
         method: bisect
Residual value: 4.6480838422624515e-07


From the abovementioned rearrangement of the force balance equations,
we can compute $F_{2}$ for a given $\theta_{1}$ in two ways:

$$
F_{2}
= \frac{-F_{1}\cos\theta_{1} - L}{\sin\theta_{2}}
= -O\cos\theta_{1} - \frac{L}{\sin\theta_{2}},
\quad \quad \quad
F_{2}
= \frac{F_{1}\sin\theta_{1} - M}{\cos\theta_{2}}
= P\sin\theta_{1} - \frac{M}{\cos\theta_{2}},
$$

which are respective consequences of summing the eastward and the northward components.

Applying the programming lesson in Part 1,
we will implement `F2_from_theta1` with a keyword argument indicating which manner of computing $F_{2}$.

In [11]:
def F2_from_theta1(
    z,  # In radians
    use_E_force_balance = True,
):
    if use_E_force_balance:
        return -(O * mt.cos(z)) - (L / mt.sin(mt.radians(theta2)))
    return (P * mt.sin(z)) - (M / mt.cos(mt.radians(theta2)))

In [12]:
print(
    "If the first cable pulls along the direction of",
    f"{Z_1} rad. ({mt.degrees(Z_1)}°) S of E,",
    "the second cable pulls with:\n"
    f"\ta {F2_from_theta1(Z_1, use_E_force_balance=True)}-kN force",
    "(according to the balance of eastward force components)\n"
    f"\ta {F2_from_theta1(Z_1, use_E_force_balance=False)}-kN force",
    "(according to the balance of northward force components)"
)

print(
    "If the first cable pulls along the direction of",
    f"{Z_2} rad. ({mt.degrees(Z_2)}°) S of E,",
    "the second cable pulls with:\n"
    f"\ta {F2_from_theta1(Z_2, use_E_force_balance=True)}-kN force",
    "(according to the balance of eastward force components)\n"
    f"\ta {F2_from_theta1(Z_2, use_E_force_balance=False)}-kN force",
    "(according to the balance of northward force components)"
)



If the first cable pulls along the direction of 3.0875779730329667 rad. (176.9051867723465°) S of E, the second cable pulls with:
	a 5.601873540119643-kN force (according to the balance of eastward force components)
	a 5.601873788140946-kN force (according to the balance of northward force components)
If the first cable pulls along the direction of -0.6441170020357027 rad. (-36.90518572926522°) S of E, the second cable pulls with:
	a -2.052424618064643-kN force (according to the balance of eastward force components)
	a -2.0524241532562586-kN force (according to the balance of northward force components)


## On computing the raw score

Full marks (100%) are granted to one who obtains:

- the abovementioned two $F_{2}$-values,
  and
- two of the abovementioned four $\theta_{1}$-values

via

- syntactically and semantically correct definitions of `residual_from_F2` and `theta1_from_F2`,
- syntactically and semantically correct definitions of `residual_from_theta1`, and `F2_from_theta1`,
  and
- syntactically correct invocations of `scipy.optimize.bisect`.

Extra marks (2%) are given for each of the following accomplishments.

- `XL_1` and `XU_1` are programmatically determined
  (*e.g.*, via `while` loops).
- `XL_2` and `XU_2` are programmatically determined
  (*e.g.*, via `while` loops).
- `ZL_1` and `ZU_1` are programmatically determined
  (*e.g.*, via `while` loops).
- `ZL_2` and `ZU_2` are programmatically determined
  (*e.g.*, via `while` loops).
- `theta1_from_F2` is implemented to allow the user to specify which force balance equation to use.
- `F2_from_theta1` is implemented to allow the user to specify which force balance equation to use.